https://geoffboeing.com/2016/11/osmnx-python-street-networks/

In [1]:
%%capture

import os, tqdm
import pandas as pd
import osmnx as ox
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points, transform
import pyproj
import heapq

# Configure OSMnx
ox.config(use_cache=True, log_console=True)

In [2]:
walking_speed = 1.22 # m/s
walking_speed = walking_speed * 60 # m/min

In [3]:
city_boxes = {'Rome': (41.9952, 41.7882, 12.6281, 12.3644), # north, south, east, west
              'Paris': (48.9520, 48.7650, 2.4900, 2.2100),
              'London': (51.6792, 51.2473, 0.2774, -0.4944)}

In [4]:
city = 'Rome, Italy'
network_type = 'all'

In [5]:
def process_graph(graph):
    # Project graph
    graph = ox.projection.project_graph(graph, to_crs=3035)

    graph = ox.simplification.consolidate_intersections(
        # Graph to simplify
        graph,
        # buffer around each node (project the graph beforehand)
        tolerance=10,
        # Get result as graph (False to get nodes only as gdf)
        rebuild_graph=True,
        # no dead ends
        dead_ends=False,
        # Reconnect (False to get intersections only)
        reconnect_edges=True
    )

    return ox.convert.to_undirected(graph)

In [6]:
bbox = city_boxes[city.split(', ')[0]]

# graph = ox.graph_from_place(city, network_type=network_type, simplify=True, retain_all=False, truncate_by_edge=True)
# graph = ox.graph_from_point((45.39842, 11.87649), dist=150, network_type=network_type)
graph = ox.graph_from_bbox(bbox = bbox, network_type=network_type)

graph = process_graph(graph)

In [7]:
%%capture
# https://wiki.openstreetmap.org/wiki/Key:amenity?uselang=en-GB

supermarket = ox.features.features_from_bbox(bbox = bbox, tags={'shop': ['supermarket','convenience']}).to_crs(epsg=3035)
pharmacy = ox.features.features_from_bbox(bbox = bbox, tags={'amenity': 'pharmacy'}).to_crs(epsg=3035)
# kindergarten = ox.features.features_from_bbox(bbox = bbox, tags={'amenity': 'kindergarten'}).to_crs(epsg=3035)
post_office = ox.features.features_from_bbox(bbox = bbox, tags={'amenity': 'post_office'}).to_crs(epsg=3035)
# doctor = ox.features.features_from_bbox(bbox = bbox, tags={'amenity': 'doctors'}).to_crs(epsg=3035)
# cafe = ox.features.features_from_bbox(bbox = bbox, tags={'amenity': ['cafe', 'bar']}).to_crs(epsg=3035)

unique_services = ['supermarket', 'pharmacy', 'post_office']

In [8]:
service_nodes = dict([(unique_services[i],[]) for i in range(len(unique_services))])

_ = [print(k, len(eval(k))) for k, v in service_nodes.items()]

supermarket 1093
pharmacy 538
post_office 174


In [9]:
# Define a function to insert a service node into the graph
def insert_service_node(graph, service_type, service_lon, service_lat):
    
    # Find the nearest edge and get the geometry to the service location
    u, v, key = ox.nearest_edges(graph, X=service_lon, Y=service_lat) # (u, v have x, y coordinates)

    # Create a point for the service location
    point = Point(service_lon, service_lat) # (lon, lat)

    # If edge is not a straight line, i.e. contains a 'geometry'
    try:
        edge_geom = graph.edges[u, v, key]['geometry']
        edge_weight = edge_geom.length
        edge_weight_linear = (edge_geom.coords[0]).distance(edge_geom.coords[-1]) * 111320

    except:
        # Coordinates of the nearest_edges
        edge_geom = LineString([(graph.nodes[u]['x'], graph.nodes[u]['y']),(graph.nodes[v]['x'], graph.nodes[v]['y'])])
        edge_weight = edge_weight_linear = Point(graph.nodes[u]['x'], graph.nodes[u]['y']).distance(Point(graph.nodes[v]['x'], graph.nodes[v]['y'])) * 111320
        # edge_weight = line_string_distance(edge_geom)
        
    # Calculate the closest point on the edge to the service location
    closest_point_on_edge = nearest_points(edge_geom, point)[0] # (lon. lat)

    # Ratio of first segment to total length
    if closest_point_on_edge == Point(edge_geom.coords[0]):
        graph.nodes[u]['service_type'] = service_type
        return
    elif closest_point_on_edge == Point(edge_geom.coords[-1]):
        graph.nodes[v]['service_type'] = service_type
        return

    # Create new edges with the split geometries
    edge_geom_1 = LineString([edge_geom.coords[0], (closest_point_on_edge.x, closest_point_on_edge.y)])
    edge_geom_2 = LineString([(closest_point_on_edge.x, closest_point_on_edge.y), edge_geom.coords[-1]])

    # Insert the new node into the graph
    new_node_id = max(graph.nodes()) + 1  # or any unique ID
    graph.add_node(new_node_id, x=closest_point_on_edge.x, y=closest_point_on_edge.y, service_type=service_type)

    ratio_segment_1 = closest_point_on_edge.distance(Point(edge_geom.coords[0])) / edge_weight_linear

    # Split weight based on ratio
    # weight_segment_1 = line_string_distance(edge_geom_1)
    # weight_segment_2 = line_string_distance(edge_geom_2)
    weight_segment_1 = edge_weight * ratio_segment_1
    weight_segment_2 = edge_weight * (1 - ratio_segment_1)

    # # Remove the old edge and add the new edges
    graph.remove_edge(u, v)
    graph.add_edge(u, new_node_id, key=key, geometry=edge_geom_1, length=weight_segment_1)
    graph.add_edge(new_node_id, v, key=key, geometry=edge_geom_2, length=weight_segment_2)

    return new_node_id

In [10]:
# x = longitude, y = latitude

for s in tqdm.tqdm(unique_services):
    if 'node' in list(eval(s).index.get_level_values(0)):
        for v in eval(s)['geometry']['node'].get_coordinates().iterrows():
            node_id = insert_service_node(graph, s, v[1]['x'], v[1]['y'])
            if node_id is not None:
                service_nodes[s].append(node_id)
    if 'way' in list(eval(s).index.get_level_values(0)):
        for v in eval(s)['geometry']['way'].get_coordinates().groupby('osmid').agg('mean').iterrows():
            node_id = insert_service_node(graph, s, v[1]['x'], v[1]['y'])
            if node_id is not None:
                service_nodes[s].append(node_id)

  0%|          | 0/3 [00:00<?, ?it/s]/Users/Marco/.pyenv/versions/3.12.0/lib/python3.12/site-packages/shapely/linear.py:203: RuntimeWarning: invalid value encountered in shortest_line
  return lib.shortest_line(a, b, **kwargs)
/Users/Marco/.pyenv/versions/3.12.0/lib/python3.12/site-packages/shapely/linear.py:203: RuntimeWarning: invalid value encountered in shortest_line
  return lib.shortest_line(a, b, **kwargs)
/Users/Marco/.pyenv/versions/3.12.0/lib/python3.12/site-packages/shapely/linear.py:203: RuntimeWarning: invalid value encountered in shortest_line
  return lib.shortest_line(a, b, **kwargs)
/Users/Marco/.pyenv/versions/3.12.0/lib/python3.12/site-packages/shapely/linear.py:203: RuntimeWarning: invalid value encountered in shortest_line
  return lib.shortest_line(a, b, **kwargs)
/Users/Marco/.pyenv/versions/3.12.0/lib/python3.12/site-packages/shapely/linear.py:203: RuntimeWarning: invalid value encountered in shortest_line
  return lib.shortest_line(a, b, **kwargs)
/Users/Marco/

In [11]:
# s = 'supermarkets'
# if 'node' in list(eval(s).index.get_level_values(0)):
#     for v in eval(s)['geometry']['node'].get_coordinates().iterrows():
#         insert_service_node(graph, s, v[1]['x'], v[1]['y'])
# if 'way' in list(eval(s).index.get_level_values(0)):
#     for v in eval(s)['geometry']['way'].get_coordinates().groupby('osmid').agg('mean').iterrows():
#         insert_service_node(graph, s, v[1]['x'], v[1]['y'])

In [12]:
# %%capture

# # for s in tqdm.tqdm(unique_services):
# for s in tqdm.tqdm(['supermarkets', 'cafe']):
#     if 'node' in list(eval(s).index.get_level_values(0)):
#         for v in eval(s)['geometry']['node'].get_coordinates().iterrows():
#             node_id = ox.nearest_nodes(graph, v[1]['x'], v[1]['y'])
#             graph.nodes[node_id]['service_type'] = s
#     if 'way' in list(eval(s).index.get_level_values(0)):
#         for v in eval(s)['geometry']['way'].get_coordinates().groupby('osmid').agg('mean').iterrows():
#             node_id = ox.nearest_nodes(graph, v[1]['x'], v[1]['y'])
#             graph.nodes[node_id]['service_type'] = s

In [13]:
for u, v, k, data in graph.edges(keys=True, data=True):
    if 'geometry' in data:
        data['weight'] = data['geometry'].length / walking_speed
    else:
        data['weight'] = data['length'] / walking_speed

In [14]:
print(f'There are {len(graph)} nodes and {len(graph.edges)} edges in the graph')

There are 41197 nodes and 74477 edges in the graph


# Dijkstra

In [15]:
def custom_dijkstra(G, source, threshold):
    # Initialize distances and visited set
    distances = {node: float('inf') for node in G.nodes}
    distances[source] = 0
    visited = set()

    # Priority queue (heap) to store nodes and their distances
    pq = [(0, source)]

    while pq:
        current_distance, current_node = heapq.heappop(pq)

        # Stop if all nodes within 15 have been visited
        if current_distance > threshold:
            break

        if current_node not in visited:
            visited.add(current_node)

        for neighbor, edge_data in G[current_node].items():
            weight = min([dict(edge_data)[k]['weight'] for k in dict(edge_data).keys()])
            new_distance = current_distance + weight
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                heapq.heappush(pq, (new_distance, neighbor))

    return visited

In [16]:
to_debug = False

def custom_dijkstra_debug(G, source, threshold):
    # Initialize distances and visited set
    distances = {node: float('inf') for node in G.nodes}
    distances[source] = 0
    visited = set()

    # Priority queue (heap) to store nodes and their distances
    pq = [(0, source)]

    print(f'Source: {source}')

    while pq:
        current_distance, current_node = heapq.heappop(pq)

        print(f'Current node: {current_node}, Current distance: {current_distance}')

        # Stop if all nodes within 15 have been visited
        if current_distance > threshold:
            print('Threshold reached, break.')
            break

        if current_node not in visited:
            print(f'Adding {current_node} to visited.')
            visited.add(current_node)

        for neighbor, edge_data in G[current_node].items():
            weight = min([dict(edge_data)[k]['weight'] for k in dict(edge_data).keys()])
            new_distance = current_distance + weight
            print(f'Neighbor: {neighbor}, weight: {weight}, new distance: {new_distance}')
            if new_distance < distances[neighbor]:
                print(f'Updating distance for {neighbor} to {new_distance}')
                distances[neighbor] = new_distance
                heapq.heappush(pq, (new_distance, neighbor))

    return visited

if to_debug:
    FMC = set(graph.nodes())
    for idx, service_type in enumerate(unique_services):
        new_node_id = max(graph.nodes) + idx + 1
        print(f'New node ID: {new_node_id}')
        graph.add_node(new_node_id) # create new node
        graph.add_weighted_edges_from([(new_node_id, node, 0) for node in service_nodes[service_type]])
        FMC_tmp = custom_dijkstra_debug(graph, new_node_id, 15)
        graph.remove_node(new_node_id)
        FMC = FMC.intersection(FMC_tmp)
        print('\n')
    print(f'There are {len(FMC)} nodes in the 15 MC')

# 15 MC

In [17]:
FMC = set(graph.nodes())
for idx, service_type in enumerate(unique_services):
    new_node_id = max(graph.nodes) + idx + 1
    graph.add_node(new_node_id) # create new node
    graph.add_weighted_edges_from([(new_node_id, node, 0) for node in service_nodes[service_type]])
    FMC_tmp = custom_dijkstra(graph, new_node_id, 15)
    graph.remove_node(new_node_id)
    FMC = FMC.intersection(FMC_tmp)
print(f'There are {len(FMC)} nodes in the 15 MC')

There are 20594 nodes in the 15 MC


In [34]:
node_c = ['w'] * len(graph.nodes)

for u in FMC:
    node_c[u] = 'r'

for u in [u for _, v in service_nodes.items() for u in v]:
    node_c[u] = 'b'

In [35]:
edge_c = []

for u, v in graph.edges():
    if u in FMC and v in FMC:
        edge_c.append('r')
    else:
        edge_c.append('k')

In [21]:
ox.plot_graph(graph, bgcolor='w', node_color=node_c, node_edgecolor='k', node_size=8, edge_color=edge_c, edge_linewidth=1, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_15MC_nodes.png'), show=False, close=True)

(<Figure size 8000x8000 with 1 Axes>, <Axes: >)

In [36]:
ox.plot_graph(graph, bgcolor='w', node_color=node_c, node_edgecolor='k', node_size=8, edge_color=edge_c, edge_linewidth=2, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_15MC_nodes_thick.png'), show=False, close=True)

(<Figure size 8000x8000 with 1 Axes>, <Axes: >)

In [37]:
edge_c2 = []

for u, v in graph.edges():
    if u in FMC and v in FMC:
        edge_c2.append('k')
    else:
        edge_c2.append('r')

In [23]:
ox.plot_graph(graph, bgcolor='w', node_size=0, edge_color=edge_c2, edge_linewidth=1, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_15MC.png'), show=False, close=True)

(<Figure size 8000x8000 with 1 Axes>, <Axes: >)

In [38]:
ox.plot_graph(graph, bgcolor='w', node_size=0, edge_color=edge_c2, edge_linewidth=2, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_15MC_thick.png'), show=False, close=True)

(<Figure size 8000x8000 with 1 Axes>, <Axes: >)

# 1-30 MC

In [24]:
max_threshold = 30
MC = [set(graph.nodes())] * max_threshold
for i in range(max_threshold):
    for idx, service_type in enumerate(unique_services):
        new_node_id = max(graph.nodes) + idx + 1
        graph.add_node(new_node_id) # create new node
        graph.add_weighted_edges_from([(new_node_id, node, 0) for node in service_nodes[service_type]])
        FMC_tmp = custom_dijkstra(graph, new_node_id, i+1)
        graph.remove_node(new_node_id)
        MC[i] = MC[i].intersection(FMC_tmp)
    print(f'There are {len(MC[i])} nodes in the {i+1} MC')

There are 2 nodes in the 1 MC
There are 210 nodes in the 2 MC
There are 890 nodes in the 3 MC
There are 1987 nodes in the 4 MC
There are 3382 nodes in the 5 MC
There are 4993 nodes in the 6 MC
There are 6760 nodes in the 7 MC
There are 8675 nodes in the 8 MC
There are 10609 nodes in the 9 MC
There are 12517 nodes in the 10 MC
There are 14367 nodes in the 11 MC
There are 16106 nodes in the 12 MC
There are 17731 nodes in the 13 MC
There are 19206 nodes in the 14 MC
There are 20594 nodes in the 15 MC
There are 21853 nodes in the 16 MC
There are 23042 nodes in the 17 MC
There are 24178 nodes in the 18 MC
There are 25169 nodes in the 19 MC
There are 26153 nodes in the 20 MC
There are 27042 nodes in the 21 MC
There are 27830 nodes in the 22 MC
There are 28608 nodes in the 23 MC
There are 29319 nodes in the 24 MC
There are 29963 nodes in the 25 MC
There are 30557 nodes in the 26 MC
There are 31063 nodes in the 27 MC
There are 31547 nodes in the 28 MC
There are 32039 nodes in the 29 MC
There a

In [25]:
MC_unique = {key + 1: MC[key].difference(*MC[:key]) for key in range(max_threshold)}

In [45]:
colours = ox.plot.get_colors(n=max_threshold+1, cmap="copper") # cool, winter, Blues

In [46]:
node_cs = [colours[max_threshold]] * len(graph.nodes)

for t in range(max_threshold - 1, -1, -1):
    for u in MC[t]:
        node_cs[u] = colours[t]

for u in [u for _, v in service_nodes.items() for u in v]:
    node_cs[u] = 'r'

In [47]:
ox.plot_graph(graph, bgcolor='w', node_color=node_cs, node_edgecolor='k', node_size=8, edge_linewidth=1, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_30tMC_nodes.png'), show=False, close=True)

(<Figure size 8000x8000 with 1 Axes>, <Axes: >)

In [48]:
ox.plot_graph(graph, bgcolor='w', node_color=node_cs, node_edgecolor='k', node_size=8, edge_linewidth=2, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_30tMC_nodes_thick.png'), show=False, close=True)

(<Figure size 8000x8000 with 1 Axes>, <Axes: >)

In [49]:
def find_key_of_value(d, number):
    for key, values in d.items():
        if number in values:
            return key
    return max(d.keys()) + 1

In [50]:
edge_cs = [colours[max_threshold]] * len(graph.edges()) # blues[max_threshold-1] darkest

for i, (u, v) in enumerate(graph.edges()):
    idx = max(find_key_of_value(MC_unique, u), find_key_of_value(MC_unique, v))
    if idx <= max_threshold:
        edge_cs[i] = colours[idx-1]

In [51]:
ox.plot_graph(graph, bgcolor='w', node_size=0, edge_color=edge_cs, edge_linewidth=1, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_30tMC.png'), show=False, close=True)

(<Figure size 8000x8000 with 1 Axes>, <Axes: >)

In [ ]:
ox.plot_graph(graph, bgcolor='w', node_size=0, edge_color=edge_cs, edge_linewidth=2, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_30tMC_thick.png'), show=False, close=True)

# 15 tMC

In [ ]:
half_max_threshold = int(max_threshold/2)

half_colours = ox.plot.get_colors(n=half_max_threshold+1, cmap="copper") # cool, winter

edge_cs = [half_colours[half_max_threshold]] * len(graph.edges())

for i, (u, v) in enumerate(graph.edges()):
    idx = max(find_key_of_value(MC_unique, u), find_key_of_value(MC_unique, v))
    if idx <= half_max_threshold:
        edge_cs[i] = half_colours[idx-1]

ox.plot_graph(graph, bgcolor='w', node_size=0, edge_color=edge_cs, edge_linewidth=1, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_15tMC.png'), show=False, close=True)

ox.plot_graph(graph, bgcolor='w', node_size=0, edge_color=edge_cs, edge_linewidth=2, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_15tMC_thick.png'), show=False, close=True)

# Save data

In [33]:
to_csv = True

os.makedirs('./data', exist_ok=True)
os.makedirs('./data/master/', exist_ok=True)

if to_csv:
    nodes, edges = ox.graph_to_gdfs(graph)

    nodes.to_csv(os.path.join('./data/master', city.split(', ')[0] + '_nodes.csv'), index=True)
    edges.to_csv(os.path.join('./data/master', city.split(', ')[0] + '_edges.csv'), index=True)

    pd.DataFrame(FMC).to_csv(os.path.join('./data/master', city.split(', ')[0] + '_FMC.csv'), index=False)
    pd.DataFrame(MC).to_csv(os.path.join('./data/master', city.split(', ')[0] + '_MC.csv'), index=False)

    edges = edges.reset_index().groupby(['u', 'v'], as_index=False).agg({'weight': 'min'}).rename(columns={'u': 'source', 'v': 'target'}, inplace=False)
    nodes = nodes.reset_index().rename(columns={'osmid': 'id', 'service_type': 'label'}, inplace=False)[['id','label']]

    nodes.to_csv(os.path.join('./data', city.split(', ')[0] + '_nodes.csv'), index=False)
    edges.to_csv(os.path.join('./data', city.split(', ')[0] + '_edges.csv'), index=False)

# Reload graph

In [ ]:
edges_df = pd.read_csv('./min_city/data/edges.csv')
nodes_df = pd.read_csv('./min_city/data/nodes.csv')

In [ ]:
graph_df = nx.from_pandas_edgelist(edges_df, 'source', 'target', ['weight'], create_using=nx.MultiDiGraph())

# Assuming your nodes_df has columns 'node', 'name', and 'gender'
nx.set_node_attributes(graph_df, pd.Series(nodes_df['label'], index=nodes_df['id']).to_dict(), 'label')

for node_id in graph_df.nodes():
    graph_df.nodes[node_id]['label'] = nodes_df.loc[node_id, 'label']

# DELETE

In [107]:
FMC_Rust = set(pd.read_csv('./min_city/output.csv', header=None))

In [108]:
node_c = ['w'] * len(graph.nodes)

for u in FMC_Rust:
    node_c[u] = 'r'

for u in [u for _, v in service_nodes.items() for u in v]:
    node_c[u] = 'b'

edge_c = []

for u, v in graph.edges():
    if u in FMC_Rust and v in FMC_Rust:
        edge_c.append('r')
    else:
        edge_c.append('k')

In [110]:
ox.plot_graph(graph, bgcolor='w', node_color=node_c, node_edgecolor='k', node_size=8, edge_color=edge_c, edge_linewidth=1, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_15_MC_nodes_Rust.png'), show=False, close=True)

(<Figure size 8000x8000 with 1 Axes>, <Axes: >)

In [111]:
edge_c2 = []

for u, v in graph.edges():
    if u in FMC_Rust and v in FMC_Rust:
        edge_c2.append('k')
    else:
        edge_c2.append('r')

In [112]:
ox.plot_graph(graph, bgcolor='w', node_size=0, edge_color=edge_c2, edge_linewidth=1, edge_alpha=1, figsize=(80,80), dpi=500, save=True, filepath=os.path.join('./images', city.split(', ')[0] + '_15_MC_Rust.png'), show=False, close=True)

(<Figure size 8000x8000 with 1 Axes>, <Axes: >)